# Experiments for Predicting Creative Mode of Thinking from EEG Signal

Importing Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
from scipy import signal
import os
import mne
import mne_microstates

Loading Sample Data

In [ ]:
f = open('FinalProjectData/sub_02.json')
data = json.load(f)

In [ ]:
data.keys()

In [ ]:
subject_dict_raw = {}
subject_dict_segments = {}

## Feature Extraction
The nature study that generated this dataset (https://www.nature.com/articles/s41598-021-81655-0) noted differences in the coverage and duration of EEG microstates, as well as the power of the upper and lower alpha band between the EEG signal between different modes of thinking. First, we will extract these features and use traditional machine learning methods to try to determine the creative mode of thinking from EEG signal

### Extracting Duration and Coverage of 6 EEG Microstates

In [ ]:
for i, file in enumerate(sorted(os.listdir('FinalProjectData'))[2:]):
    f = open('FinalProjectData/' + file)
    data = json.load(f)
    subject_dict_raw[i] = {}
    subject_dict_segments[i] = {}
    for key in list(data.keys()):
        arr = np.array(data[key])
        subject_dict_raw[i][key] = arr
        raw = mne.io.RawArray(arr, info = mne.create_info([str(i) for i in range(arr.shape[0])], 300, ch_types='eeg', verbose=0), verbose=0)
        raw.set_eeg_reference('average', verbose=0)
        raw.filter(0.2, None, verbose=0)
        raw.pick_types(meg=False, eeg=True, verbose=0)
        maps, segmentation = mne_microstates.segment(raw.get_data(), n_states=6, verbose=0)
        subject_dict_segments[i][key] = segmentation
    #     mne_microstates.plot_segmentation(segmentation[:500], raw.get_data()[:, :500], raw.times[:500])

In [ ]:
microstate_rel_frequency = np.zeros((28, 4, 6, 1))

for subject in list(subject_dict_segments.keys()):
    for key in subject_dict_segments[subject].keys():
        if 'rest' in key:
            i=0
        elif 'idea generation' in key:
            i=1
        elif 'evolution' in key:
            i=2
        else:
            i=3
        arr = subject_dict_segments[subject][key]
        
        for n in np.unique(arr):
            if microstate_rel_frequency[subject][i][n] == 0: 
                microstate_rel_frequency[subject][i][n] = (arr == n).sum()/len(arr)
            else:
                microstate_rel_frequency[subject][i][n] += (arr == n).sum()/len(arr)

Visualizing the average durations of the 6 microstates across subjects for the different modes of thinking

In [ ]:
mean_over_all_subjects = np.average(microstate_rel_frequency, axis=0)

In [ ]:
mean_over_all_subjects[1:, :] = mean_over_all_subjects[1:, :]/3 
mean_over_all_subjects[0, :] = mean_over_all_subjects[0, :]/2

In [ ]:
keys = ['Rest', 'Idea Generation', 'Idea Evolution', 'Evaluation']

In [ ]:
for ms in range(6):
    plt.scatter(keys, mean_over_all_subjects[:, ms])
    plt.title(f"Microstate {ms+1}")
    plt.ylabel('Relative Frequency')
    plt.show()
    # plt.savefig('Microstate_' + str(ms+1) + '.png')


Plotting the microstates

In [ ]:
arr = subject_dict_raw[0]['1_idea evolution']
raw = mne.io.RawArray(arr, info = mne.create_info([str(i) for i in range(arr.shape[0])], 300, ch_types='eeg', verbose=None))
maps, segmentation = mne_microstates.segment(raw.get_data(), n_states=6, verbose=0)
mne_microstates.plot_segmentation(segmentation[:600],raw.get_data()[:, :600], raw.times[:600])

In [ ]:
plt.plot(raw.times[:600], raw.get_data()[1, :600] )

Training a model to only use the microstate features to predict mode of thinking

In [ ]:
X = microstate_rel_frequency.reshape((112,6))
y = keys * 28

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=265)

In [ ]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

svm = SVC(C=.20)
svm = MLPClassifier((20, 50, 65, 20,), max_iter=1000, verbose=False)
svm.fit(X_train, y_train)

In [ ]:
preds = svm.predict(X_test)
accuracy = (preds==y_test).sum()/len(preds)
accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=svm.classes_)
disp.plot()
plt.savefig('confusion_matrix_just_microstate.png')

### Adding Frequency Domain Features (Upper and Lower Alpha Band Power)

In [ ]:
from scipy.fft import fft, fftfreq
import seaborn as sns

In [ ]:
sig = subject_dict_raw[0]['1_idea generation'][0]
sampling_frequency = 300
time = np.arange(sig.size) / sampling_frequency

fig, ax = plt.subplots(1, 1, figsize=(12,4))
plt.plot(time, sig, lw=1.5, color='k')
plt.xlabel('Time (Seconds)')
plt.ylabel('Voltage')
plt.xlim([time.min(), time.max()])
plt.title('Single Channel EEG Data')
sns.despine()
# plt.plot([i for i in range(sig[0].size)], sig[0], color='k')

In [ ]:
from scipy import signal
size = len(sig)
fs = 300

F, PSD = signal.welch(sig, fs, nperseg=size)
plt.plot(F,PSD, color='k')
plt.title("Welch's Periodogram")
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power [$\mu V^2$/Hz]')
# plt.ylim([0,40])
plt.xlim([0,20])
plt.show()

In [ ]:
import seaborn as sns
low, high = 8, 10

# Find intersecting values in frequency vector
idx_alpha = np.logical_and(F >= low, F <= high)

# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(F, PSD, lw=2, color='k')
plt.fill_between(F, PSD, where=idx_alpha, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([6, 14])
# plt.ylim([0, PSD.max() * 1.1])
plt.ylim([0,.1e-5])
plt.title("Welch's Periodogram")
sns.despine()

In [ ]:
from scipy.integrate import simps

# Frequency resolution
freq_res = F[1] - F[0]

# Compute the absolute power by approximating the area under the curve
alpha_power = simps(PSD[idx_alpha], dx=freq_res)
print('Absolute lower alpha power: %.10f uV^2' % alpha_power)

In [ ]:
subject_dict_raw[0]['1_idea evolution'].shape

In [ ]:
trp_alpha = np.zeros((28, 4, 2, 1))

for subject in list(subject_dict_raw.keys()):
    curr_subject = subject_dict_raw[subject]
    for key in list(curr_subject.keys()):
        if 'rest' in key:
            i=0
        elif 'idea generation' in key:
            i=1
        elif 'evolution' in key:
            i=2
        else:
            i=3
        sig = curr_subject[key]
        sig = np.mean(arr, axis=0)
        size = len(sig)
        fs = 300
        F, PSD = signal.welch(sig, fs, nperseg=size)
        
        low, high = 8, 10
        # Find intersecting values in frequency vector
        idx_lower_alpha = np.logical_and(F >= low, F <= high)
        req_res = F[1] - F[0]
        lower_alpha_power = simps(PSD[idx_lower_alpha], dx=freq_res)
        trp_alpha[subject][i][0] = lower_alpha_power
        
        low, high = 10, 12
        # Find intersecting values in frequency vector
        idx_upper_alpha = np.logical_and(F >= low, F <= high)
        req_res = F[1] - F[0]
        upper_alpha_power = simps(PSD[idx_upper_alpha], dx=freq_res)        
        trp_alpha[subject][i][1] = upper_alpha_power 

In [ ]:
trp_alpha_all_channels = np.zeros((28, 63, 4, 2, 1))

for subject in list(subject_dict_raw.keys()):
    curr_subject = subject_dict_raw[subject]
    for key in list(curr_subject.keys()):
        if 'rest' in key:
            i=0
        elif 'idea generation' in key:
            i=1
        elif 'evolution' in key:
            i=2
        else:
            i=3
        sig = curr_subject[key]
#         sig = np.mean(arr, axis=0)
        for channel in range(sig.shape[0]):
            curr = sig[channel]
            size = len(curr)
            fs = 300
            F, PSD = signal.welch(curr, fs, nperseg=size)

            low, high = 8, 10
            # Find intersecting values in frequency vector
            idx_lower_alpha = np.logical_and(F >= low, F <= high)
            req_res = F[1] - F[0]
            lower_alpha_power = simps(PSD[idx_lower_alpha], dx=freq_res)
            trp_alpha_all_channels[subject][channel][i][0] = lower_alpha_power

            low, high = 10, 12
            # Find intersecting values in frequency vector
            idx_upper_alpha = np.logical_and(F >= low, F <= high)
            req_res = F[1] - F[0]
            upper_alpha_power = simps(PSD[idx_upper_alpha], dx=freq_res)        
            trp_alpha_all_channels[subject][channel][i][1] = upper_alpha_power 

Plotting the average trp alpha values across all subjects for the different modes of thinking

In [ ]:
across_subject_alpha = np.average(trp_alpha, axis=0)
across_subject_alpha.shape

In [ ]:
plt.title('Absolute Lower and Upper Alpha Power')
plt.ylabel('Absolute Power')
plt.scatter(keys, mean_over_all_subjects[:, 0], color='blue', label='Average Lower Alpha Power')
plt.scatter(keys, mean_over_all_subjects[:, 1], color='orange', label="Average Upper Alpha Power")
plt.legend()
plt.savefig('alpha_power.png')

In [ ]:
trp_alpha.shape

In [ ]:
microstate_rel_frequency.shape

In [ ]:
microstate_and_alpha = np.append(microstate_rel_frequency, trp_alpha, axis=2).reshape(28,4,8,1)

### Modeling with Microstate Features and Frequency Domain Features

In [ ]:
X = microstate_and_alpha.reshape((112,8))
y = keys * 28
y = np.array(y)

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X)

In [ ]:
from tensorflow.keras.utils import to_categorical

numeric_labels = []
for lab in y:
    if 'Evolution' in lab:
        numeric_labels.append(0)
    elif 'Generation' in lab:
        numeric_labels.append(1)
    elif 'Evaluation' in lab:
        numeric_labels.append(2)
    else:
        numeric_labels.append(3)
        
y = to_categorical(numeric_labels)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=.3, random_state=265)

In [ ]:
import tensorflow.keras as keras
model = keras.models.Sequential()
model.add(keras.layers.Dense(64, activation='tanh', input_shape=(8,)))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(128, activation='tanh'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(256, activation='tanh'))
model.add(keras.layers.Dense(4, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.00005)
model.compile(optimizer=opt, loss='CategoricalCrossentropy', metrics=['accuracy'])

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit(X_train, y_train, epochs = 1000, batch_size=10, validation_data=(X_test, y_test), callbacks=[callback, model_checkpoint_callback])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
test_loss = model.evaluate(X_test, y_test)
test_loss

In [ ]:
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, 'orange', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy Loss')
plt.legend()
plt.savefig('network_loss_history')

In [ ]:
from sklearn.svm import SVC
y = keys * 28
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=.3, random_state=265)

lr = SVC(C=100, max_iter=10000, kernel="poly", degree=2, random_state=11)
lr.fit(X_train, y_train)

In [ ]:
preds = lr.predict(X_test)
accuracy = (preds==y_test).sum()/len(preds)
accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=svm.classes_)
disp.plot()
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=265)
pca_results = pca.fit_transform(X_scaled)

In [ ]:
sns.scatterplot(x=pca_results[:,0], y=pca_results[:,1], hue=y)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('PCA Representation of Data')
plt.savefig('pca_scatter.png')

In [ ]:
from mlxtend.plotting import plot_decision_regions
svm_pca = SVC(C=100, random_state=265, kernel='poly', degree=4)
svm_pca.fit(pca_results, y)
plot_decision_regions(np.array(pca_results), y, axis=1, clf=svm_pca, legend=3)
plt.title('SVM Polynomial Kernel Decision Boundary')
plt.xlabel('PCA Dimension 1')
plt.ylabel('PCA Dimension 2')
plt.savefig('SVM_decision_boundary.png')

## Converting the EEG Signal to EEG Spectrograms

In [ ]:
import cv2

def generate_spectrogram(data, sampling_frequency=300):
    data = np.mean(data, axis=0)
    f, t, Sxx = signal.spectrogram(
        data,
        fs=sampling_frequency,
        nperseg=sampling_frequency,
        noverlap=sampling_frequency*0.95,)    
    plt.ylim([0,20])
    plt.axis('off')
    plt.pcolormesh(t, f, Sxx, cmap='viridis')
    plt.savefig('temp', bbox_inches='tight', pad_inches=0)
    img = cv2.imread('temp.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img, img.shape

In [ ]:
labels = []
spectrograms = []
subjects = []

for subject in list(subject_dict_raw.keys()):
    curr_subject = subject_dict_raw[subject]
    for key in list(curr_subject.keys()):
        curr = curr_subject[key]
        curr = np.array(curr)
        for i in range(3600, curr.size, 3600):
            clip = curr[:, i-3600:i]
            if clip.shape[1] == 3600:
                spect, shape = generate_spectrogram(clip)
                spectrograms.append(spect)
                if 'evolution' in key:
                    labels.append("Idea Evolution")
                elif 'generation' in key:
                    labels.append("Idea Gemeration")
                elif 'rating' in key:
                    labels.append('Idea Evaluation')
                else:
                    labels.append('Rest')
                subjects.append(subject)
                

In [ ]:
X = np.array(spectrograms)
y = np.array(labels)
X.shape

In [ ]:
# np.save('Spectrograms_new', X)
# np.save('Labels_new', y)

Creating the datasets and oversampling

In [ ]:
import numpy as np

In [ ]:
X = np.load('Spectrograms.npy')
y = np.load('Labels.npy')
X.shape

In [ ]:
numeric_labels = []
for lab in y:
    if 'Evolution' in lab:
        numeric_labels.append(0)
    elif 'Gemeration' in lab:
        numeric_labels.append(1)
    elif 'Evaluation' in lab:
        numeric_labels.append(2)
    else:
        numeric_labels.append(3)

y = np.array(numeric_labels)

In [ ]:
evolution = y==0
generation = y==1
evaluation = y==2
rest = y==3

In [ ]:
evolution

In [ ]:
y_evolution = y[evolution]
y_generation = y[generation]
y_evaluation = y[evaluation]
y_rest = y[rest]

In [ ]:
X_evolution = X[evolution]
X_generation = X[generation]
X_evalutation = X[evaluation]
X_rest = X[rest]

In [ ]:
evolution_sample = np.random.randint(0, len(y[evolution]), 600)
generation_sample = np.random.randint(0, len(y[generation]), 600)
evaluation_sample = np.random.randint(0, len(y[evaluation]), 600)
rest_sample = np.random.randint(0, len(y[rest]), 600)

In [ ]:
X_bal_evolution = X_evolution[evolution_sample]
y_bal_evolution = y_evolution[evolution_sample]

X_bal_generation = X_generation[generation_sample]
y_bal_generation = y_generation[generation_sample]

X_bal_evaluation = X_evalutation[evaluation_sample]
y_bal_evaluation = y_evaluation[evaluation_sample]

X_bal_rest = X_rest[rest_sample]
y_bal_rest = y_rest[rest_sample]

In [ ]:
X_balanced = np.concatenate([X_bal_evolution, X_bal_generation, X_bal_evaluation, X_bal_rest])
y_balanced = np.concatenate([y_bal_evolution, y_bal_generation, y_bal_evaluation, y_bal_rest])

In [ ]:
from tensorflow.keras.utils import to_categorical
y_balanced = to_categorical(y_balanced)

In [ ]:
validation_idx = np.random.randint(0, len(y_balanced), 300)
validationX = X_balanced[validation_idx]
validationY = y_balanced[validation_idx]
trainX = np.delete(X_balanced, validation_idx, 0)
trainY = np.delete(y_balanced, validation_idx, 0)

In [ ]:
np.unique(trainY, axis=0)

Defining our Convolutional Neural Network Model to Process the Spectrograms

In [ ]:
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam, SGD

model = Sequential()
model.add(Conv2D(16, 3, activation='relu', padding='same' ,input_shape=(217, 334, 3)))
model.add(Conv2D(16, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(3))
model.add(Conv2D(32, 3, activation='relu', padding='same'))
model.add(Conv2D(32, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
# model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Training the model and including data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0, horizontal_flip=True)
it_train = datagen.flow(trainX, trainY, batch_size=64)
steps = int(trainX.shape[0] / 64)
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(it_train, steps_per_epoch=steps, epochs=200, validation_data=(validationX, validationY), callbacks=[callback, model_checkpoint_callback], verbose=1)

Plotting loss and accuracy

In [ ]:
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'][1:], color='blue', label='Training Loss')
plt.plot(history.history['val_loss'][1:], color='orange', label='Validation Loss')
plt.title('CNN Loss History')
plt.legend()
plt.savefig('CNN loss.png')

In [ ]:
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('CNN Accuracy History')
plt.legend()
plt.savefig('CNN Accuracy.png')

In [ ]:
_, acc = model.evaluate(validationX, validationY, verbose=0)
acc

In [ ]:
preds = model(validationX)
preds = np.argmax(preds,axis=1)

In [ ]:
truth= np.argmax(validationY, axis=1)

Displaying a confusion matrrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(truth, preds)
cm

In [ ]:
disp = ConfusionMatrixDisplay(cm, display_labels=['Idea Evolution', 'Idea Generation', 'Idea Evaluation', 'Rest'])
disp.plot()
plt.savefig('CNN Spectrogram Test Confusion Matrix')

## 1D Convolution

In [ ]:
model = Sequential()
model.add(Conv2D(16, 3, activation='relu', padding='same', input_shape=(217, 334, 3)))
model.add(MaxPooling2D(3))
model.add(Conv2D(16, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(3))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))
opt = Adam(learning_rate=.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0, horizontal_flip=True)
it_train = datagen.flow(trainX, trainY, batch_size=64)
steps = int(trainX.shape[0] / 64)
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(it_train, steps_per_epoch=steps, epochs=200, validation_data=(validationX, validationY), callbacks=[callback, model_checkpoint_callback], verbose=1)

In [ ]:
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'][1:], color='blue', label='Training Loss')
plt.plot(history.history['val_loss'][1:], color='orange', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('CNN 1D Loss.png')


In [ ]:
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('CNN 1D Accuracy.png')

In [ ]:
_, acc = model.evaluate(validationX, validationY, verbose=0)
acc

In [ ]:
preds = model(validationX)
preds = np.argmax(preds,axis=1)

In [ ]:
truth= np.argmax(validationY, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(truth, preds)
cm

In [ ]:
disp = ConfusionMatrixDisplay(cm, display_labels=['Idea Evolution', 'Idea Generation', 'Idea Evaluation', 'Rest'])
disp.plot()
plt.savefig('CNN Spectrogram Test Confusion Matrix')

### Using GradCAM to Visualize Parts of the Spectrograms that Contribute to the Prediction of each Class

In [ ]:
import tensorflow as tf

preprocess_input = keras.applications.xception.preprocess_input


def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=1):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
img_array = preprocess_input(get_img_array('temp.png', size=(217, 334, 3)))

heatmap = make_gradcam_heatmap(img_array, model, 'conv2d_23')

In [ ]:
plt.matshow(heatmap)
plt.axis('off')
plt.savefig('Gradcam Idea Generation', bbox_inches='tight', pad_inches=0)

In [ ]:
from IPython.display import Image, display
import matplotlib.cm as cm

def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.3):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))


save_and_display_gradcam('temp.png', heatmap)

In [ ]:
model(trainX[200:202])

In [ ]:
trainY[200:202]

In [ ]:
time = np.arange(dat[0].size) / 300
plt.plot(time, dat[0])
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.savefig('signal.png')

In [ ]:
dat = np.array(data['1_idea generation'])[:, :3600]
generate_spectrogram(dat)

In [ ]:
dat.size